In [2]:
# 라이브러리 불러오기
import os
import pandas as pd
import numpy as np
import time

In [3]:
df_test = pd.read_csv('C:/Users/user/Desktop/GA4_project/train.csv')
# 다음과 같은 경고는 column에 NaN값이나 여러 type데이터가 섞여 있을 때 나타나는 경고
# 경고대로 dtpye을 명시해주거나 low_memory = False를 입력하면 된다. 혹은 불러온 다음에 데이터를 다룰 때 참고사항으로 해두면 된다고 함
# low_memory 옵션, 
# 대용량의 데이터를 불러오는 경우 각 칼럼의 데이터 타입(dtype)을 추측하는 것이 매우 많은 메모리를 사용하기 때문에
# 대용량의 데이터를 불러올때 메모리 에러가 발생하는 경우 이를 False로 설정할 것을 권장한다.

C:\Users\user\AppData\Local\Temp\ipykernel_4156\2866665893.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('C:/Users/user/Desktop/GA4_project/train.csv')


In [4]:
df = pd.read_csv('C:/Users/user/Desktop/GA4_project/train.csv', low_memory = False)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   channelGrouping       903653 non-null  object
 1   date                  903653 non-null  int64 
 2   device                903653 non-null  object
 3   fullVisitorId         903653 non-null  object
 4   geoNetwork            903653 non-null  object
 5   sessionId             903653 non-null  object
 6   socialEngagementType  903653 non-null  object
 7   totals                903653 non-null  object
 8   trafficSource         903653 non-null  object
 9   visitId               903653 non-null  int64 
 10  visitNumber           903653 non-null  int64 
 11  visitStartTime        903653 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 82.7+ MB


- **channelGrouping** : 사이트에 들어온 경로
- **fullVisitorId**  : 유저별로 그룹화 할 때 사용 가능
- **date** : 사이트 방문일(날짜 데이터)
- **device** : 접속한 브라우저 명
- **geoNetwork** : 접속 지역
- **totals** : 페이지 방문 횟수 등 로그 데이터
- **trafficSource** : 세션이 시작된 트래픽 출처
- **visitNumber** : 같은 유저의 방문 횟수 누적 표시
- **visitStartTime** : 방문 시작 시간(초단위)(Posix time)

In [6]:
# 해당 값 전체 보기
# pd.set_option('display.max_colwidth', None)

# 출력 옵션 초기화 (선택사항)
# pd.reset_option('display.max_colwidth')

In [7]:
# pd.reset_option('display.max_colwidth')
df.tail(3)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
903650,Social,20170104,"{""browser"": ""Android Webview"", ""browserVersion...",5744576632396406899,"{""continent"": ""Asia"", ""subContinent"": ""Eastern...",5744576632396406899_1483526434,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""24"", ""pageviews"": ""21...","{""referralPath"": ""/yt/about/ko/"", ""campaign"": ...",1483526434,1,1483526434
903651,Social,20170104,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",2709355455991750775,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",2709355455991750775_1483592857,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""24"", ""pageviews"": ""22...","{""referralPath"": ""/l.php"", ""campaign"": ""(not s...",1483592857,1,1483592864
903652,Social,20170104,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",0814900163617805053,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",0814900163617805053_1483574474,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""31"", ""pageviews"": ""31...","{""referralPath"": ""/yt/about/es-419/"", ""campaig...",1483574474,1,1483574474


In [8]:
# 해당 값 전체 보기
# pd.set_option('display.max_colwidth', None)
print(df[['device']][:2])
print(df[['totals']][:2])
print(df[['trafficSource']][:2])
# 열 안의 값들이 json 형식으로 입력되어 있음
# 어떻게 분리해서 볼 수 있을지 고민하다가 다른 분들은 어떻게 했는지 참고했더니
# 결국 데이터를 분석하기 위해서는 이를 분리할 필요가 있다.

                                              device
0  {"browser": "Chrome", "browserVersion": "not a...
1  {"browser": "Firefox", "browserVersion": "not ...
                                              totals
0  {"visits": "1", "hits": "1", "pageviews": "1",...
1  {"visits": "1", "hits": "1", "pageviews": "1",...
                                       trafficSource
0  {"campaign": "(not set)", "source": "google", ...
1  {"campaign": "(not set)", "source": "google", ...


- json형식으로 입력된 열들을 분리시켜 다시 df 불러오기 (방법 : kaggle 다른 분의 코드 참고)

In [9]:
import os 
import pandas as pd
import numpy as np
import json #json 형식 다루는 라이브러리
from pandas import json_normalize

def load_df(csv_path='C:/Users/user/Desktop/GA4_project/train.csv', nrows=None): # None은 모든 행의 데이터를 불러오기를 뜻함
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important! 부분에서 str형식으로 바꿔서 보라고 언급
                     nrows=nrows)
    # converters : 각 column에 해당하는 값에 json.loads 함수를 실행하여 다른 형태로 변환하여 불러온다는 의미 
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column]) # json_normalize : JSON 데이터를 평평한(flat) 테이블 형태로 변환해주는 함수
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [10]:
#train_df = load_df()

In [11]:
#train_df.columns

- POSIX시간을 UTC시간으로 바꾸는 작업

In [12]:
# 데이터프레임에 한 열의 값을 바꾸기 위해서 def함수를 만들고 apply 적용
import pandas as pd
import datetime #날짜와 시간을 다루는 라이브러리
import pytz #다양한 시간대를 처리하기 위해 사용하는 라이브러리

# Posix 시간을 utc 시간으로 변환하는 함수
def posix_to_utc(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime)
    return utc_time

# Posix 시간을 UTC 지정한 로컬 시간대로 변환하는 함수
def posix_to_utc_local(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime) #posix -> utc시간대로
    local_timezone = pytz.timezone('America/New_York')  # 지역 시간대
    return utc_time.astimezone(local_timezone) # 그 지역 시간대로 변환


In [13]:
df_test['utc_local_time'] = df_test['visitStartTime'].apply(posix_to_utc)

In [14]:
df_test[['date','utc_local_time']]

,date,utc_local_time
0,20160902,2016-09-02 15:33:05
1,20160902,2016-09-03 05:22:27
2,20160902,2016-09-03 01:16:26
3,20160902,2016-09-03 05:40:13
4,20160902,2016-09-02 13:23:20
...,...,...
903648,20170104,2017-01-04 18:32:30
903649,20170104,2017-01-04 15:29:58
903650,20170104,2017-01-04 10:40:34
903651,20170104,2017-01-05 05:07:44


In [ ]:
proproccessed_df = 